In [8]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
import faiss
import numpy as np

class SiameseKoBERT(nn.Module):
    def __init__(self):
        super(SiameseKoBERT, self).__init__()
        self.tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
        self.bert = BertModel.from_pretrained('monologg/kobert')
        self.fc = nn.Linear(self.bert.config.hidden_size, self.bert.config.hidden_size)

    def forward_one(self, x):
        inputs = self.tokenizer(x, return_tensors="pt", truncation=True, padding=True, max_length=128)
        outputs = self.bert(**inputs)
        return self.fc(outputs['last_hidden_state'][:, 0, :])

    def forward(self, x1, x2):
        out1 = self.forward_one(x1)
        out2 = self.forward_one(x2)
        return out1, out2

model = SiameseKoBERT()

# Define a contrastive loss
...

# Train your model using pairs (X, Y)
...


Ellipsis

In [13]:
import pandas as pd
file_path = '../crawling/file_content_list.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)
df = df.dropna(subset=['Content'])
Y_texts = df['Content']
X_texts = df['Title']

In [7]:
# Embed Y texts

Y_vectors = [model.forward_one(y_text).detach().numpy() for y_text in Y_texts]
Y_vectors = np.vstack(Y_vectors)

# Build the Faiss index
d = Y_vectors.shape[1]
index = faiss.IndexFlatL2(d)
index.add(Y_vectors)

In [28]:
class TextPairDataset(Dataset):
    def __init__(self, X_texts, Y_texts):
        self.X_texts = X_texts
        self.Y_texts = Y_texts
    
    def __len__(self):
        return len(self.X_texts)
    
    def __getitem__(self, idx):
        return self.X_texts[idx], self.Y_texts[idx]

# Define the DataLoader
batch_size = 32
dataset = TextPairDataset(X_texts, Y_texts)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

print(dataloader.__len__())


29


In [30]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = nn.functional.pairwise_distance(output1, output2, keepdim=True)
        loss_contrastive = torch.mean((1 - label) * torch.pow(euclidean_distance, 2) + 
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss_contrastive

# Define the device, model, optimizer, and loss
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SiameseKoBERT().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
criterion = ContrastiveLoss()



In [32]:
# Training loop
n_epochs = 10
for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    try:
        for x, y in dataloader:
        
            optimizer.zero_grad()
            x, y = x.to(device), y.to(device)
            output1, output2 = model(x, y)
            
            # Assuming labels are 1 for each pair since they are matching pairs
            labels = torch.ones(batch_size, 1).to(device)
            
            loss = criterion(output1, output2, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
    except KeyError:
            print(f"Error at batch index: {i}")
    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {total_loss/len(dataloader)}")


KeyError: 609